# Spark and Data Science Libraries Sample Notebook

This notebook demonstrates how to use PySpark in conjunction with pandas, numpy, scikit-learn and LightGBM for data analysis and machine learning tasks.

Created: May 7, 2025

## 1. Setting up the Spark Session

In [1]:
import os
import pyspark
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("SparkDataScienceSample") \
    .master(os.environ.get("SPARK_MASTER", "spark://spark-master:7077")) \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark UI available at: {spark.sparkContext.uiWebUrl}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 00:53:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.4.1
Spark UI available at: http://0020415e2355:4040


## 2. Creating Sample Data with PySpark

In [2]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import random

# Create a schema for our data
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("category", StringType(), False),
    StructField("value1", DoubleType(), False),
    StructField("value2", DoubleType(), False),
    StructField("target", IntegerType(), False)
])

# Generate some random data
categories = ["A", "B", "C", "D"]
data = []

for i in range(1000):
    category = random.choice(categories)
    value1 = random.uniform(0, 100)
    value2 = random.uniform(0, 100)
    # Simple rule: if value1 > 50 and category is A or B, target is 1, otherwise 0
    target = 1 if (value1 > 50 and category in ["A", "B"]) else 0
    data.append(Row(id=i, category=category, value1=value1, value2=value2, target=target))

# Create a DataFrame
df = spark.createDataFrame(data)

# Show a sample of the data
df.show(5)
print(f"Total records: {df.count()}")

# Show basic statistics
df.describe().show()

25/05/08 00:53:51 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.18.0.5 executor 0): java.io.IOException: Cannot run program "/opt/conda/bin/python": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:222)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:134)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPa

Py4JJavaError: An error occurred while calling o47.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.18.0.3 executor 1): java.io.IOException: Cannot run program "/opt/conda/bin/python": error=2, No such file or directory
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:222)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:134)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	

## 3. Converting Spark DataFrame to Pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Display pandas DataFrame info
print("Pandas DataFrame Information:")
pandas_df.info()

# Basic statistics with pandas
print("\nBasic Statistics:")
print(pandas_df.describe())

# Display correlation between numeric columns
correlation = pandas_df[["value1", "value2", "target"]].corr()
print("\nCorrelation Matrix:")
print(correlation)

## 4. Data Visualization

In [ ]:
# Set the style for plots
plt.figure(figsize=(12, 10))
sns.set(style="whitegrid")

# Create a subplot grid
plt.subplot(2, 2, 1)
sns.histplot(pandas_df["value1"], kde=True)
plt.title("Distribution of value1")

plt.subplot(2, 2, 2)
sns.countplot(x="category", data=pandas_df)
plt.title("Count by Category")

plt.subplot(2, 2, 3)
sns.scatterplot(x="value1", y="value2", hue="target", data=pandas_df)
plt.title("Scatter Plot of value1 vs value2 by target")

plt.subplot(2, 2, 4)
sns.boxplot(x="category", y="value1", data=pandas_df)
plt.title("value1 by Category")

plt.tight_layout()
plt.show()

## 5. Feature Engineering

In [ ]:
# Let's add some features using numpy and pandas
pandas_df["value_ratio"] = pandas_df["value1"] / (pandas_df["value2"] + 1)  # Adding 1 to avoid division by zero
pandas_df["value_sum"] = pandas_df["value1"] + pandas_df["value2"]
pandas_df["value_diff"] = np.abs(pandas_df["value1"] - pandas_df["value2"])
pandas_df["value1_log"] = np.log1p(pandas_df["value1"])  # log1p to avoid issues with zero

# Create dummy variables for category
category_dummies = pd.get_dummies(pandas_df["category"], prefix="category")
pandas_df = pd.concat([pandas_df, category_dummies], axis=1)

# Display the DataFrame with new features
print("DataFrame with new features:")
pandas_df.head()

## 6. Machine Learning with Scikit-Learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Select features for model training
features = ["value1", "value2", "value_ratio", "value_sum", "value_diff", "value1_log", 
            "category_A", "category_B", "category_C", "category_D"]
X = pandas_df[features]
y = pandas_df["target"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rf_model.predict(X_test_scaled)
y_prob = rf_model.predict_proba(X_test_scaled)[:, 1]

# Evaluate the model
print("Random Forest Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    "Feature": features,
    "Importance": rf_model.feature_importances_
}).sort_values("Importance", ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x="Importance", y="Feature", data=feature_importance)
plt.title("Feature Importance from Random Forest")
plt.tight_layout()
plt.show()

## 7. Advanced ML with LightGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

# Create LightGBM datasets
lgb_train = lgb.Dataset(X_train_scaled, y_train)
lgb_test = lgb.Dataset(X_test_scaled, y_test, reference=lgb_train)

# Set parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Train model
print("Training LightGBM model...")
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=100,
                      valid_sets=[lgb_train, lgb_test],
                      early_stopping_rounds=10)

# Make predictions
lgb_pred = lgb_model.predict(X_test_scaled)
lgb_binary_pred = [1 if pred > 0.5 else 0 for pred in lgb_pred]

# Evaluate the model
print("\nLightGBM Performance:")
print(f"Accuracy: {accuracy_score(y_test, lgb_binary_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, lgb_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, lgb_binary_pred))

# Plot feature importance
plt.figure(figsize=(10, 6))
lgb.plot_importance(lgb_model, max_num_features=10)
plt.title("Feature Importance from LightGBM")
plt.tight_layout()
plt.show()

## 8. Going Back to Spark - Saving Results to Spark DataFrame

In [ ]:
# Add predictions to our pandas DataFrame
test_indices = pandas_df.iloc[y_test.index].index
pandas_df.loc[test_indices, "rf_prediction"] = y_pred
pandas_df.loc[test_indices, "rf_probability"] = y_prob
pandas_df.loc[test_indices, "lgb_prediction"] = lgb_binary_pred
pandas_df.loc[test_indices, "lgb_probability"] = lgb_pred

# Convert back to a Spark DataFrame
result_spark_df = spark.createDataFrame(pandas_df)
result_spark_df.show(5)

# Register as a temporary view for SQL queries
result_spark_df.createOrReplaceTempView("model_results")

# Example: Run a Spark SQL query
print("\nAnalyzing model results using Spark SQL:")
spark.sql("""
SELECT 
    category,
    COUNT(*) as total_count,
    SUM(CASE WHEN target = 1 THEN 1 ELSE 0 END) as actual_positive,
    SUM(CASE WHEN rf_prediction = 1 THEN 1 ELSE 0 END) as rf_predicted_positive,
    SUM(CASE WHEN lgb_prediction = 1 THEN 1 ELSE 0 END) as lgb_predicted_positive
FROM model_results
GROUP BY category
ORDER BY category
""").show()

## 9. Close Spark Session when Done

In [ ]:
# Stop the Spark session
# Uncomment this when you want to close the Spark session
# spark.stop()

print("Notebook completed successfully!")